In [1]:
# Utilities
import json

# Pull data from API
from lib.api_request import *

# Webscraping 
from lib.webscraper import *

### Collect the **core dataset** from the D&D5E API

The bulk of our dataset will be fetched from the 'www.dnd5eapi.co' API. This includes information of the various races, subraces and classes available in the Basic Rules, along other elements such as alignments and ability scores.

In [2]:
# Getting urls for the various races, subraces and classes
races_url = get_url('races')
classes_url = get_url('classes')
subraces_url = get_url('subraces')

In [3]:
# Getting a list of alignments and ability scores
alignments = get_elements('alignments')
ability_scores = get_elements('ability-scores')

In [6]:
# Collect race, subrace and class data in the form of dictionaries
races_dataset = get_race_data(races_url)
classes_dataset = get_class_data(classes_url)
subraces_dataset = get_subrace_data(subraces_url)

### Add in some data from 5E Player's Handbook for **character flavor**

Basic Rules only encompass a single subrace and subclass (resp. per race and class). We want our generator to create a character based in the Player's Handbook, so some additional data will be added manually here.

In [10]:
# Subclasses

phb_5e_subclasses = {
    'Barbarian' : ['Berserker', 'Totem Warrior'],
    'Bard' : ['College of Lore', 'College of Valor'],
    'Cleric' : ['Death', 'Knowledge', 'Life', 'Light', 'Nature', 'Tempest', 'Trickery', 'War'],
    'Druid' : ['Land', 'Moon'],
    'Fighter' : ['Battle Master', 'Champion', 'Eldritch knight'],
    'Monk' : ['Four Elements', 'Open Hand', 'Shadow'],
    'Paladin' : ['Oath of the Ancients',  'Oath of Devotion', 'Oath of Vengeance'],
    'Ranger' : ['Beast Master', 'Hunter'],
    'Rogue' : ['Arcane trickster', 'Assassin', 'Thief'],
    'Sorcerer' : ['Draconic Bloodline', 'Wild Mage'],
    'Warlock' : ['Archfey', 'Fiend', 'Great Old One'],
    'Wizard' : ['Abjurer', 'Conjurer', 'Diviner', 'Enchanter', 'Invoker', 'Illusioner', 'Necromancer', 'Transmuter']
}

In [11]:
# Update the class dataset with new subclasses
for c1 in classes_dataset:
    for c2 in phb_5e_subclasses:
        if c1 == c2:
            classes_dataset[c1]['subclasses'] = list(set(classes_dataset[c1]['subclasses'] + phb_5e_subclasses[c2]))

In [12]:
# Check subclasses
for classname, attributes in classes_dataset.items():
    print(classname, attributes['subclasses'])
    

Barbarian ['Berserker', 'Totem Warrior']
Bard ['College of Lore', 'Lore', 'College of Valor']
Cleric ['Nature', 'Trickery', 'Death', 'Life', 'Light', 'Tempest', 'War', 'Knowledge']
Druid ['Moon', 'Land']
Fighter ['Champion', 'Eldritch knight', 'Battle Master']
Monk ['Open Hand', 'Four Elements', 'Shadow']
Paladin ['Oath of the Ancients', 'Oath of Vengeance', 'Oath of Devotion', 'Devotion']
Ranger ['Hunter', 'Beast Master']
Rogue ['Arcane trickster', 'Assassin', 'Thief']
Sorcerer ['Wild Mage', 'Draconic Bloodline', 'Draconic']
Warlock ['Archfey', 'Fiend', 'Great Old One']
Wizard ['Necromancer', 'Conjurer', 'Abjurer', 'Invoker', 'Diviner', 'Evocation', 'Transmuter', 'Enchanter', 'Illusioner']


In [13]:
# Remove duplicates

classes_dataset['Bard']['subclasses'].remove('Lore') # Duplicate of 'College of Lore'
classes_dataset['Paladin']['subclasses'].remove('Devotion') # Duplicate of 'Oath of Devotion'
classes_dataset['Sorcerer']['subclasses'].remove('Draconic') # Duplicate of 'Draconic Bloodline'
classes_dataset['Wizard']['subclasses'].remove('Evocation') # Duplicate of 'Invoker'

In [14]:
# Subraces
phb_5e_subraces = {
    'Dwarf': ['Hill Dwarf', 'Mountain Dwarf'],
    'Elf': ['High Elf', 'Wood Elf', 'Drow'],
    'Gnome': ['Rock Gnome', 'Forest Gnome'],
    'Halfling': ['Lightfoot Halfling', 'Stout Halfling'],
}

In [15]:
# Update the race dataset with new subraces
for r1 in races_dataset:
    for r2 in phb_5e_subraces:
        if r1 == r2:
            races_dataset[r1]['subraces'] = list(set(races_dataset[r1]['subraces'] + phb_5e_subraces[r2]))

In [16]:
# Update the subraces dataset with information from the PHB

phb_5e_additional_subraces = {
    'Wood Elf' : {
        'racial_traits' : ['Elf Weapon Training','Fleet of Foot','Mask of the Wild'],
        'starting_proficiencies' : ['Longswords', 'Shortswords', 'Shortbows', 'Longbows'],
        'ability_bonus' : {'WIS':1},
        'race' : 'Elf'
    },
    'Drow' : {
        'racial_traits' : ['Superior Darkvision', 'Sunlight Sensitivity', 'Drow Magic', 'Drow Weapon Training'],
        'starting_proficiencies' : ['Rapiers', 'Shortswords', 'Hand crossbows'],
        'ability_bonus' : {'CHA':1},
        'race' : 'Elf'
    },
    'Mountain Dwarf' : {
        'racial_traits' : ['Dwarven Armor Training'],
        'starting_proficiencies' : ['Light Armor', 'Medium Armor'],
        'ability_bonus' : {'STR':2},
        'race' : 'Dwarf'
    },
    'Forest Gnome' : {
        'racial_traits' : ['Natural Illustionist', 'Speak With Small Beasts'],
        'starting_proficiencies' : [],
        'ability_bonus' : {'DEX': 1},
        'race' : 'Gnome'
    },
    'Stout Halfling' : {
        'racial_traits' : ['Stout Resilience'],
        'starting_proficiencies' : [],
        'ability_bonus' : {'CON':1},
        'race' : 'Halfling'
    },
}


In [17]:
# Update the subrace dataset with new subraces attributes
subraces_dataset.update(phb_5e_additional_subraces)

### Collect additional data via **webscraping**

For roleplaying purposes, we will gather information on the many Pantheons and deities of Faêrun, by scraping it from a long time favorite wiki of mine: Forgotten Realms Wiki, available at https://forgottenrealms.fandom.com/wiki/.

In [19]:
#Select only Pantheons that are relevant to the playable races of the PHB.

keep_pantheons = [
        'Human deities',
        'Lords of the Golden Hills',
        'Morndinsamman',
        "Yondalla's Children",
        'Orcish pantheon',
        'Seldarine',
        'Dark Seldarine'
        # Add desired 'Category:'' items
    ]         

In [20]:
# Collect data (grab yourself a coffee, this can take a while)
deities_url = get_deity_url(keep_pantheons)
deities_dataset = get_deity_details(deities_url)

In [2]:
# Checking the list of deities for each pantheon
for pantheon in deities_dataset.keys():
    print(pantheon, deities_dataset[pantheon].keys())

In [26]:
# Drop Kozah (which is equivalent to Talos)
deities_dataset['Human deities'].pop('Kozah')

# Drop detities from ancient pantheons (Pharaonic / Mulhorandi...) 
deities_dataset['Human deities'].pop('Anubis')
deities_dataset['Human deities'].pop('Apep')
deities_dataset['Human deities'].pop('Nephthys')
deities_dataset['Human deities'].pop('Poseidon')

# Fix the format for Mystra (Midnight)
deities_dataset['Human deities']['Mystra'] = deities_dataset['Human deities'].pop('Mystra (Midnight)')

### Export dataset

We'll use a JSON format to store our collected dataset.

In [28]:
# Put the complete dataset together
dataset = {
    'races' : races_dataset,
    'subraces' : subraces_dataset,
    'classes' : classes_dataset,
    'alignments' : alignments,
    'ability_scores' : ability_scores,
    'deities': deities_dataset
}

In [29]:
# Dump it to a JSON file
with open('./data/dataset.json','w') as json_file:
    json.dump(dataset, json_file)